In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime

def create_tuple(response):
    response_data = response.json()
    response_datum = response_data[0]
    response_target = response_datum['target']
    response_datapoints = response_datum['datapoints']
    tuple_array = [tuple(x) for x in response_datapoints]
    npa = np.array(tuple_array, dtype=[
        ('value', np.double), ('ts', 'datetime64[ms]')])
    return npa

def query(target, server, start, end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query" #WebCTRL
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
            "additional": [
                    "noagg",
                ]
          },
          "target": target
        }
      ],
    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)


def query_to_list(point, server, start, end):
    master = query(point, server, start, end)

    list = pd.Series(data=[i[0] for i in master], index=[i[1] for i in master])
    # print("\n", point, "\n", list)

    list = list[~list.index.duplicated()]
    # print("\n", point, " new\n", list)

    return list

In [15]:
# Gates G01

query_to_list(point="#g01_vav-g-1/occ1", server="gates_main", start=str(datetime(2023, 1, 15)), end=str(datetime.now()))

<Response [400]>


KeyError: 0

In [16]:
# Phillips 101

query_to_list(point="#phillips_lecture_hall_101/occ1", server="duffield_main", start=str(datetime(2023, 1, 15)), end=str(datetime.now()))

<Response [200]>


2023-01-16 12:59:25.130    1.0
2023-01-16 13:00:24.030    1.0
2023-01-16 13:40:43.840    0.0
2023-01-16 13:41:43.020    0.0
2023-01-17 09:25:57.550    1.0
                          ... 
2024-03-01 11:46:09.220    1.0
2024-03-01 12:01:43.150    0.0
2024-03-01 12:02:42.140    0.0
2024-03-01 13:34:16.340    1.0
2024-03-01 13:35:16.130    1.0
Length: 6602, dtype: float64